In [13]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_fighter_fight_urls(fighter_url):
    page = requests.get(fighter_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    fight_urls = []
    
    fight_rows = soup.find_all('tr', class_='b-fight-details__table-row')
    for fight_row in fight_rows:
        onclick_value = fight_row.get('onclick')
        if onclick_value:
            match = re.search(r"'(http://www.ufcstats.com/fight-details/.+?)'", onclick_value)
            if match:
                fight_link = match.group(1)
                fight_urls.append(fight_link)
    
    return fight_urls


In [14]:
fight_url = 'http://www.ufcstats.com/fight-details/e3aad51099a23ba4'

In [29]:
def scrape_fight_data(fight_url):
    fight_page = requests.get(fight_url)
    fight_soup = BeautifulSoup(fight_page.content, 'html.parser')

    data = []
    fight_table = fight_soup.find('tbody', class_='b-fight-details__table-body')
    
    fighter_divs = fight_soup.find_all('div', class_='b-fight-details__person')
    outcomes = [outcome_tag.get_text(strip=True) if (outcome_tag := fighter_div.find('i', class_='b-fight-details__person-status')) else None for fighter_div in fighter_divs]

    for idx, row in enumerate(fight_table.find_all('tr', class_='b-fight-details__table-row')):
        fighter_names = row.find_all('a', class_='b-link_style_black')
        fighters = [name.get_text(strip=True) for name in fighter_names]
        
        kd = [col.get_text(strip=True) for col in row.find_all('td')[1].find_all('p')]
        sig_str = [col.get_text(strip=True) for col in row.find_all('td')[2].find_all('p')]
        sig_str_percent = [col.get_text(strip=True) for col in row.find_all('td')[3].find_all('p')]
        total_str = [col.get_text(strip=True) for col in row.find_all('td')[4].find_all('p')]
        td = [col.get_text(strip=True) for col in row.find_all('td')[5].find_all('p')]
        td_percent = [col.get_text(strip=True) for col in row.find_all('td')[6].find_all('p')]
        sub_att = [col.get_text(strip=True) for col in row.find_all('td')[7].find_all('p')]
        rev = [col.get_text(strip=True) for col in row.find_all('td')[8].find_all('p')]
        ctrl = [col.get_text(strip=True) for col in row.find_all('td')[9].find_all('p')]

        for i in range(len(fighters)):
            fight_info = {
                'Fighter': fighters[i],
                'KD': kd[i],
                'Sig. str.': sig_str[i],
                'Sig. str. %': sig_str_percent[i],
                'Total str.': total_str[i],
                'Td': td[i],
                'Td %': td_percent[i],
                'Sub. att': sub_att[i],
                'Rev.': rev[i],
                'Ctrl': ctrl[i],
                'Fight URL': fight_url,
                'Outcome': outcomes[i]  # Use the outcome for this fighter
            }
            data.append(fight_info)

    df = pd.DataFrame(data)

    new_data = []
    new_columns = [
        'Fighter', 'Opponent', 'KD', 'Opponent KD', 'Sig. str.', 'Opponent Sig. str.',
        'Sig. str. %', 'Opponent Sig. str. %', 'Total str.', 'Opponent Total str.',
        'Td', 'Opponent Td', 'Td %', 'Opponent Td %', 'Sub. att', 'Opponent Sub. att',
        'Rev.', 'Opponent Rev.', 'Ctrl', 'Opponent Ctrl', 'Fight URL', 'Outcome'
    ]

    # Iterate through the original DataFrame to extract fighter and opponent stats
    for i in range(0, len(df), 2):
        fighter_row = df.iloc[i]
        opponent_row = df.iloc[i + 1]

        new_row = [
            fighter_row['Fighter'], opponent_row['Fighter'],
            fighter_row['KD'], opponent_row['KD'],
            fighter_row['Sig. str.'], opponent_row['Sig. str.'],
            fighter_row['Sig. str. %'], opponent_row['Sig. str. %'],
            fighter_row['Total str.'], opponent_row['Total str.'],
            fighter_row['Td'], opponent_row['Td'],
            fighter_row['Td %'], opponent_row['Td %'],
            fighter_row['Sub. att'], opponent_row['Sub. att'],
            fighter_row['Rev.'], opponent_row['Rev.'],
            fighter_row['Ctrl'], opponent_row['Ctrl'],
            fighter_row['Fight URL'], fighter_row['Outcome']
        ]

        new_data.append(new_row)

    new_df = pd.DataFrame(new_data, columns=new_columns)
    return new_df


scrape_fight_data(fight_url)
scrape_fight_data('http://www.ufcstats.com/fight-details/04f32bb0c0d3a8cc')


,Fighter,Opponent,KD,Opponent KD,Sig. str.,Opponent Sig. str.,Sig. str. %,Opponent Sig. str. %,Total str.,Opponent Total str.,...,Td %,Opponent Td %,Sub. att,Opponent Sub. att,Rev.,Opponent Rev.,Ctrl,Opponent Ctrl,Fight URL,Outcome
0,Israel Adesanya,Alex Pereira,0,0,86 of 162,91 of 157,53%,57%,119 of 209,140 of 214,...,25%,100%,0,0,0,0,6:34,0:31,http://www.ufcstats.com/fight-details/04f32bb0...,L


In [ ]:
import re
import requests
from bs4 import BeautifulSoup

    
fighter_url = 'http://www.ufcstats.com/fighter-details/029eaff01e6bb8f0'

fighter_page = requests.get(fighter_url)
fighter_soup = BeautifulSoup(fighter_page.content, 'html.parser')

fighter_name = fighter_soup.find('span', class_='b-content__title-highlight').get_text(strip=True)

print(fighter_name)

Dustin Poirier


In [73]:

all_fight_dfs = []
for fight_url in fight_urls:
    fight_df = scrape_fight_data(fight_url)
    all_fight_dfs.append(fight_df)

combined_df = pd.concat(all_fight_dfs, ignore_index=True)
combined_df

,Fighter,Opponent,KD,Opponent KD,Sig. str.,Opponent Sig. str.,Sig. str. %,Opponent Sig. str. %,Total str.,Opponent Total str.,...,Td %,Opponent Td %,Sub. att,Opponent Sub. att,Rev.,Opponent Rev.,Ctrl,Opponent Ctrl,Fight URL,Outcome
0,Dustin Poirier,Justin Gaethje,0,1,27 of 52,41 of 66,51%,62%,27 of 52,41 of 66,...,---,---,0,0,0,0,0:00,0:00,http://www.ufcstats.com/fight-details/c27e33ee...,L
1,Dustin Poirier,Michael Chandler,0,0,38 of 76,43 of 100,50%,43%,63 of 105,65 of 135,...,---,42%,2,0,0,0,0:47,5:39,http://www.ufcstats.com/fight-details/4bf3010f...,W
2,Charles Oliveira,Dustin Poirier,0,1,73 of 131,58 of 94,55%,61%,98 of 157,69 of 105,...,0%,---,3,0,1,0,5:41,0:35,http://www.ufcstats.com/fight-details/b22eab3a...,W
3,Dustin Poirier,Conor McGregor,0,0,36 of 66,27 of 38,54%,71%,36 of 66,43 of 54,...,50%,---,0,1,0,0,3:18,0:00,http://www.ufcstats.com/fight-details/7d0ce28c...,W
4,Dustin Poirier,Conor McGregor,1,0,48 of 91,29 of 66,52%,43%,53 of 98,52 of 92,...,100%,0%,0,0,0,0,1:42,1:07,http://www.ufcstats.com/fight-details/3d3e7dc7...,W
5,Dustin Poirier,Dan Hooker,0,0,153 of 231,155 of 253,66%,61%,208 of 289,182 of 283,...,14%,44%,4,0,0,0,2:49,6:02,http://www.ufcstats.com/fight-details/f8dd1e75...,W
6,Khabib Nurmagomedov,Dustin Poirier,0,0,22 of 41,12 of 39,53%,30%,54 of 77,30 of 57,...,87%,---,2,1,0,0,8:52,0:00,http://www.ufcstats.com/fight-details/01a4827b...,W
7,Max Holloway,Dustin Poirier,0,0,181 of 446,178 of 381,40%,46%,208 of 473,180 of 383,...,---,12%,1,0,0,0,0:06,3:35,http://www.ufcstats.com/fight-details/87126427...,L
8,Eddie Alvarez,Dustin Poirier,0,1,26 of 76,54 of 115,34%,46%,35 of 88,76 of 138,...,0%,---,1,2,0,0,2:28,0:05,http://www.ufcstats.com/fight-details/bd1fc4ff...,L
9,Dustin Poirier,Justin Gaethje,0,0,174 of 351,115 of 212,49%,54%,179 of 357,116 of 213,...,0%,---,0,0,0,0,0:06,1:04,http://www.ufcstats.com/fight-details/bfae2fb5...,W


In [75]:
fighter_name = 'Dustin Poirier'

for index, row in combined_df.iterrows():
    if row['Opponent'] == fighter_name:
        combined_df.at[index, ['Fighter', 'Opponent']] = row['Opponent'], row['Fighter']
        combined_df.at[index, ['KD', 'Opponent KD']] = row['Opponent KD'], row['KD']
        combined_df.at[index, ['Sig. str.', 'Opponent Sig. str.']] = row['Opponent Sig. str.'], row['Sig. str.']
        combined_df.at[index, ['Sig. str. %', 'Opponent Sig. str. %']] = row['Opponent Sig. str. %'], row['Sig. str. %']
        combined_df.at[index, ['Total str.', 'Opponent Total str.']] = row['Opponent Total str.'], row['Total str.']
        combined_df.at[index, ['Td', 'Opponent Td']] = row['Opponent Td'], row['Td']
        combined_df.at[index, ['Td %', 'Opponent Td %']] = row['Opponent Td %'], row['Td %']
        combined_df.at[index, ['Sub. att', 'Opponent Sub. att']] = row['Opponent Sub. att'], row['Sub. att']
        combined_df.at[index, ['Rev.', 'Opponent Rev.']] = row['Opponent Rev.'], row['Rev.']
        combined_df.at[index, ['Ctrl', 'Opponent Ctrl']] = row['Opponent Ctrl'], row['Ctrl']
        
        # Swap outcome values 'W' and 'L'
        if row['Outcome'] == 'W':
            combined_df.at[index, 'Outcome'] = 'L'
        elif row['Outcome'] == 'L':
            combined_df.at[index, 'Outcome'] = 'W'


combined_df

,Fighter,Opponent,KD,Opponent KD,Sig. str.,Opponent Sig. str.,Sig. str. %,Opponent Sig. str. %,Total str.,Opponent Total str.,...,Td %,Opponent Td %,Sub. att,Opponent Sub. att,Rev.,Opponent Rev.,Ctrl,Opponent Ctrl,Fight URL,Outcome
0,Dustin Poirier,Justin Gaethje,0,1,27 of 52,41 of 66,51%,62%,27 of 52,41 of 66,...,---,---,0,0,0,0,0:00,0:00,http://www.ufcstats.com/fight-details/c27e33ee...,L
1,Dustin Poirier,Michael Chandler,0,0,38 of 76,43 of 100,50%,43%,63 of 105,65 of 135,...,---,42%,2,0,0,0,0:47,5:39,http://www.ufcstats.com/fight-details/4bf3010f...,W
2,Dustin Poirier,Charles Oliveira,1,0,58 of 94,73 of 131,61%,55%,69 of 105,98 of 157,...,---,0%,0,3,0,1,0:35,5:41,http://www.ufcstats.com/fight-details/b22eab3a...,L
3,Dustin Poirier,Conor McGregor,0,0,36 of 66,27 of 38,54%,71%,36 of 66,43 of 54,...,50%,---,0,1,0,0,3:18,0:00,http://www.ufcstats.com/fight-details/7d0ce28c...,W
4,Dustin Poirier,Conor McGregor,1,0,48 of 91,29 of 66,52%,43%,53 of 98,52 of 92,...,100%,0%,0,0,0,0,1:42,1:07,http://www.ufcstats.com/fight-details/3d3e7dc7...,W
5,Dustin Poirier,Dan Hooker,0,0,153 of 231,155 of 253,66%,61%,208 of 289,182 of 283,...,14%,44%,4,0,0,0,2:49,6:02,http://www.ufcstats.com/fight-details/f8dd1e75...,W
6,Dustin Poirier,Khabib Nurmagomedov,0,0,12 of 39,22 of 41,30%,53%,30 of 57,54 of 77,...,---,87%,1,2,0,0,0:00,8:52,http://www.ufcstats.com/fight-details/01a4827b...,L
7,Dustin Poirier,Max Holloway,0,0,178 of 381,181 of 446,46%,40%,180 of 383,208 of 473,...,12%,---,0,1,0,0,3:35,0:06,http://www.ufcstats.com/fight-details/87126427...,W
8,Dustin Poirier,Eddie Alvarez,1,0,54 of 115,26 of 76,46%,34%,76 of 138,35 of 88,...,---,0%,2,1,0,0,0:05,2:28,http://www.ufcstats.com/fight-details/bd1fc4ff...,W
9,Dustin Poirier,Justin Gaethje,0,0,174 of 351,115 of 212,49%,54%,179 of 357,116 of 213,...,0%,---,0,0,0,0,0:06,1:04,http://www.ufcstats.com/fight-details/bfae2fb5...,W


In [80]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def past_fights(fighter_url):
    fighter_page = requests.get(fighter_url)
    fighter_soup = BeautifulSoup(fighter_page.content, 'html.parser')

    fighter_name = fighter_soup.find('span', class_='b-content__title-highlight').get_text(strip=True)
    fight_urls = get_fighter_fight_urls(fighter_url)
    all_fight_dfs = []

    for fight_url in fight_urls:
        fight_df = scrape_fight_data(fight_url)
        all_fight_dfs.append(fight_df)

    combined_df = pd.concat(all_fight_dfs, ignore_index=True)

    for index, row in combined_df.iterrows():
        if row['Opponent'] == fighter_name:
            combined_df.at[index, ['Fighter', 'Opponent']] = row['Opponent'], row['Fighter']
            combined_df.at[index, ['KD', 'Opponent KD']] = row['Opponent KD'], row['KD']
            combined_df.at[index, ['Sig. str.', 'Opponent Sig. str.']] = row['Opponent Sig. str.'], row['Sig. str.']
            combined_df.at[index, ['Sig. str. %', 'Opponent Sig. str. %']] = row['Opponent Sig. str. %'], row['Sig. str. %']
            combined_df.at[index, ['Total str.', 'Opponent Total str.']] = row['Opponent Total str.'], row['Total str.']
            combined_df.at[index, ['Td', 'Opponent Td']] = row['Opponent Td'], row['Td']
            combined_df.at[index, ['Td %', 'Opponent Td %']] = row['Opponent Td %'], row['Td %']
            combined_df.at[index, ['Sub. att', 'Opponent Sub. att']] = row['Opponent Sub. att'], row['Sub. att']
            combined_df.at[index, ['Rev.', 'Opponent Rev.']] = row['Opponent Rev.'], row['Rev.']
            combined_df.at[index, ['Ctrl', 'Opponent Ctrl']] = row['Opponent Ctrl'], row['Ctrl']

            if row['Outcome'] == 'W':
                combined_df.at[index, 'Outcome'] = 'L'
            elif row['Outcome'] == 'L':
                combined_df.at[index, 'Outcome'] = 'W'

    return combined_df

fighter_url = 'http://www.ufcstats.com/fighter-details/1338e2c7480bdf9e'
combined_data = past_fights(fighter_url)



In [81]:
combined_data

,Fighter,Opponent,KD,Opponent KD,Sig. str.,Opponent Sig. str.,Sig. str. %,Opponent Sig. str. %,Total str.,Opponent Total str.,...,Td %,Opponent Td %,Sub. att,Opponent Sub. att,Rev.,Opponent Rev.,Ctrl,Opponent Ctrl,Fight URL,Outcome
0,Israel Adesanya,Alex Pereira,1,0,41 of 96,49 of 85,42%,57%,41 of 96,49 of 85,...,---,---,0,0,0,0,0:03,0:00,http://www.ufcstats.com/fight-details/fe143cb8...,W
1,Israel Adesanya,Alex Pereira,0,0,86 of 162,91 of 157,53%,57%,119 of 209,140 of 214,...,25%,100%,0,0,0,0,6:34,0:31,http://www.ufcstats.com/fight-details/04f32bb0...,L
2,Israel Adesanya,Jared Cannonier,0,0,116 of 230,90 of 157,50%,57%,163 of 277,141 of 217,...,---,0%,0,0,0,0,0:00,4:14,http://www.ufcstats.com/fight-details/a7eaf7b1...,W
3,Israel Adesanya,Robert Whittaker,1,0,79 of 169,59 of 136,46%,43%,98 of 188,74 of 151,...,---,40%,0,0,0,0,0:00,3:40,http://www.ufcstats.com/fight-details/ca8f73d0...,W
4,Israel Adesanya,Marvin Vettori,0,0,96 of 181,58 of 176,53%,32%,122 of 208,91 of 211,...,---,28%,0,0,2,0,0:36,6:55,http://www.ufcstats.com/fight-details/6cd44e1b...,W
5,Israel Adesanya,Jan Blachowicz,0,0,78 of 161,107 of 193,48%,55%,99 of 182,184 of 276,...,---,60%,0,0,0,0,0:01,7:06,http://www.ufcstats.com/fight-details/7de9867b...,L
6,Israel Adesanya,Paulo Costa,1,0,55 of 84,12 of 26,65%,46%,55 of 84,12 of 26,...,---,---,0,0,0,0,0:09,0:01,http://www.ufcstats.com/fight-details/fd76b75c...,W
7,Israel Adesanya,Yoel Romero,0,0,48 of 132,40 of 89,36%,44%,48 of 132,40 of 89,...,---,0%,0,0,0,0,0:00,0:00,http://www.ufcstats.com/fight-details/b46f2f00...,W
8,Israel Adesanya,Robert Whittaker,2,0,40 of 95,32 of 116,42%,27%,40 of 95,32 of 116,...,---,---,0,0,0,0,0:04,0:00,http://www.ufcstats.com/fight-details/2556b752...,W
9,Israel Adesanya,Kelvin Gastelum,4,0,109 of 237,91 of 220,45%,41%,119 of 247,92 of 222,...,---,11%,2,0,0,1,0:42,1:21,http://www.ufcstats.com/fight-details/9bfda7c2...,W
